In [42]:
import pandas as pd

In [43]:
df_estoque = pd.read_excel("bases/relatorio_produtos.xlsx")

In [44]:
print(f"Formato dos dados {df_estoque.shape}")

Formato dos dados (37527, 10)


In [45]:
print(df_estoque.columns.to_list())

['Código', 'Produto', 'Embalagem', 'Unidade', 'Código da Marca', 'Marca', 'Quantidade estoque', 'Preço de custo', 'Código da categoria', 'Categoria']


In [46]:
df_estoque.dtypes

Código                   int64
Produto                 object
Embalagem               object
Unidade                 object
Código da Marca          int64
Marca                   object
Quantidade estoque     float64
Preço de custo         float64
Código da categoria      int64
Categoria               object
dtype: object

In [47]:
df_estoque.describe()

,Código,Código da Marca,Quantidade estoque,Preço de custo,Código da categoria
count,37527.000000,37527.000000,31204.000000,37527.000000,37527.000000
mean,19170.185413,4116.673302,112.105966,22.896258,2661.045034
std,11023.203749,19900.846543,1310.514860,431.449346,8705.304106
min,1.000000,1.000000,-4.000000,0.000000,1.000000
25%,9616.500000,995.000000,0.000000,2.524401,83.000000
50%,19186.000000,2712.000000,0.000000,6.880567,322.000000
75%,28728.500000,7346.000000,20.000000,16.858232,647.000000
max,38230.000000,901455.000000,166112.000000,79000.000000,35157.000000


In [48]:
df_estoque.describe(include=object)

,Produto,Embalagem,Unidade,Marca,Categoria
count,37527,37527,37527,37527,37527
unique,36637,358,23,872,705
top,CUECA AM BOXER LUPO,UN/1,UN,MARIA GENI DISTRIB. DE ART. DE PAP. LTDA,CADERNOS ( CAD )
freq,84,23289,23749,2685,3797


In [49]:
print("Quantidade valores faltantes\n")
df_estoque.isnull().sum()

Quantidade valores faltantes



Código                    0
Produto                   0
Embalagem                 0
Unidade                   0
Código da Marca           0
Marca                     0
Quantidade estoque     6323
Preço de custo            0
Código da categoria       0
Categoria                 0
dtype: int64

In [50]:
print("'%' dos valores faltantes")
((df_estoque.isnull().sum()/df_estoque['Código'].count() * 100).round(2))

'%' dos valores faltantes


Código                  0.00
Produto                 0.00
Embalagem               0.00
Unidade                 0.00
Código da Marca         0.00
Marca                   0.00
Quantidade estoque     16.85
Preço de custo          0.00
Código da categoria     0.00
Categoria               0.00
dtype: float64

In [51]:
df_estoque = df_estoque.rename(columns={'Código': 'Código Produto'})

In [52]:
df_estoque['Quantidade estoque'] = df_estoque['Quantidade estoque'].fillna(0)

In [53]:
df_produto = df_estoque.drop(columns=['Embalagem', 'Unidade'], axis=1).copy()

In [54]:
codigos_produtos = df_estoque['Código Produto'].nunique()
produtos_unicos = df_estoque['Produto'].nunique()
print(f"{codigos_produtos} Código de Produtos Únicos")
print(f"{produtos_unicos} Produtos Únicos")

37527 Código de Produtos Únicos
36637 Produtos Únicos


In [55]:
cod_cat_unicos = df_estoque['Código da categoria'].nunique()
cat_unicas = df_estoque['Categoria'].nunique()
print(f"{cod_cat_unicos} Código de Categorias Únicas")
print(f"{cat_unicas} Categorias Únicas")


747 Código de Categorias Únicas
705 Categorias Únicas


In [56]:
cod_marc = df_estoque['Código da Marca'].nunique()
marc_uni = df_estoque['Marca'].nunique()
print(f"{cod_marc} Código de Marcas Únicas")
print(f"{marc_uni} Marcas Únicas")


892 Código de Marcas Únicas
872 Marcas Únicas


In [57]:
produtos_duplicados = df_estoque[df_estoque['Produto'].duplicated(keep='first')]
produtos_duplicados[['Código Produto', 'Produto']].reset_index(drop=True)

,Código Produto,Produto
0,44,COPO 200ML BR C/100 ULTRA
1,342,CAPA ENCAD A4 CORES ALAPLAST
2,345,CAPA ENCAD OF CORES C/100 PEUPACK
3,565,APAGADOR QDO BR CORTIARTE
4,744,REAB MARC QDO BR 20ML PT GRAMPLINE
...,...,...
885,37860,BOB PEAD PIC 20X30 C/500 SEGPLAST
886,37884,GARRAFA FUN STITCH 600ML PLASUTIL
887,38012,MARCA TEXTO LUMINI AMARELO CIS
888,38032,BOB TERMICA AM 48G 57X15 C/54 DML THEGA


In [58]:
repetidos = produtos_duplicados['Produto'].nunique()
total_cod = df_estoque['Código Produto'].count()
print(f"{repetidos} Produtos repetidos {(repetidos/total_cod) * 100:.2f} %")

386 Produtos repetidos 1.03 %


In [59]:
df_produto = df_produto.drop_duplicates(keep='first', subset='Produto')

In [60]:
repetidos = produtos_duplicados['Marca'].nunique()
total_cod = df_estoque['Código da Marca'].count()
print(f"{repetidos} Produtos repetidos {(repetidos/total_cod) * 100:.2f} %")

114 Produtos repetidos 0.30 %


In [61]:
repetidos = produtos_duplicados['Categoria'].nunique()
total_cod = df_estoque['Código da categoria'].count()
print(f"{repetidos} Produtos repetidos {(repetidos/total_cod) * 100:.2f} %")

123 Produtos repetidos 0.33 %


In [62]:
df_produto.nunique(axis=0)

Código Produto         36637
Produto                36637
Código da Marca          891
Marca                    871
Quantidade estoque      1406
Preço de custo         25550
Código da categoria      746
Categoria                704
dtype: int64

In [63]:
print(f"Valores nulos em 'Código da Marca': {df_produto['Código da Marca'].isnull().sum()}")
print(f"Valores nulos em 'Marca': {df_produto['Marca'].isnull().sum()}")

Valores nulos em 'Código da Marca': 0
Valores nulos em 'Marca': 0


In [64]:
# Agrupar por código e contar marcas distintas
inconsistencias = df_produto.groupby('Código da Marca')['Marca'].nunique()
codigos_problema = inconsistencias[inconsistencias > 1].index

print(f"\nCódigos com mais de uma marca associada: {len(codigos_problema)}")
print(df_produto[df_produto['Código da Marca'].isin(codigos_problema)].sort_values('Código da Marca'))


Códigos com mais de uma marca associada: 0
Empty DataFrame
Columns: [Código Produto, Produto, Código da Marca, Marca, Quantidade estoque, Preço de custo, Código da categoria, Categoria]
Index: []


In [65]:
# Agrupar por marca e contar códigos distintos
marcas_multiplos_codigos = df_produto.groupby('Marca')['Código da Marca'].nunique()
marcas_problema = marcas_multiplos_codigos[marcas_multiplos_codigos > 1].index.tolist()

print(f"\nMarcas com mais de um código associado: {marcas_problema}\n")
print(f"Total: {len(marcas_problema)} marcas")


Marcas com mais de um código associado: ['ABC DISTRIBUIDORA S/A', 'AC COMERCIO VAREJISTA DE MATERIAL DE ESCRITORIO LTDA', 'ADERE PRODUTOS AUTO ADESIVOS LTDA', 'BIC AMAZONIA S/A', 'COMERCIAL IBIAPINA LTDA', 'CONDOR PINCEIS LTDA', 'ELGIN S A', 'FC OLIVEIRA E CIA LTDA', 'FORT-PLAST INDUSTRIA E COMERCIO DE PLASTICOS EIRELI', 'GERARDOS DISTRIBUIDORA LTDA', 'HENKEL LTDA', 'LUME INDUSTRIA QUIMICA E COMERCIO LTDA', 'MARLUVAS EQUIPAMENTOS DE SEGURANCA LTDA', 'MELHORAMENTOS CMPC LTDA', 'MERCORIENTAL IMPORTACAO E EXPORTACAO LTDA', 'OWENS ILLINOIS DO BRASIL IND E COM S.A', 'PLASUTIL INDUSTRIA E COMERCIO DE PLASTICOS LTDA', 'SOCIEDADE DISTRIBUIDORA DO NORDESTE LTDA', 'SOLMAR DISTRIBUIDORA DE ALIMENTOS LTDA', 'SUZANO PAPEL E CELULOSE SA']

Total: 20 marcas


In [66]:
# Criar um dicionário de mapeamento {marca: código_unico}
codigos_por_marca = df_produto.groupby('Marca')['Código da Marca'].first().to_dict()

# Substituir os códigos originais pelo código único
df_produto['Código da Marca'] = df_produto['Marca'].map(codigos_por_marca)

In [67]:
df_produto

,Código Produto,Produto,Código da Marca,Marca,Quantidade estoque,Preço de custo,Código da categoria,Categoria
0,1,LAPIS Nº2 HB PT C/144 SERELEPE ECOLE,1,MARIA GENI DISTRIB. DE ART. DE PAP. LTDA,0.0,9.000000,708,LAPIS ( LAP )
1,3,CANETA HIDRO PONTA FINA PT CIS,1272,COMERCIO E IMPORTACAO SERTIC LTDA,0.0,1.480000,35011,CANETA HIDROGRAFICA (CNT)
2,4,BANDEIRINHAS JUNINAS MEGA,1,MARIA GENI DISTRIB. DE ART. DE PAP. LTDA,0.0,2.739700,359,BANDEIRAS ( BDJ )
3,5,FITA IMP EPSON FX890 MASTERPRINT,4281,GRUPO MASTER COMERCIO IMPORTACAO E EXPORTACAO ...,61.0,10.545270,3,FITAS PARA IMPRESSORAS ( FTA )
4,6,ACUCAR COMUM,1,MARIA GENI DISTRIB. DE ART. DE PAP. LTDA,0.0,2.500000,199,AÇÚCAR
...,...,...,...,...,...,...,...,...
37522,38226,PISTOLA P/APLICAR COLA QUENTE PQ FUTURO,8793,FUTURO BRASIL IMPORTACAO E EXPORTACAO LTDA EPP,0.0,0.000000,206,PISTOLA ( PIS )
37523,38227,PERFURADOR METAL 2F 20FL FUTURO,8793,FUTURO BRASIL IMPORTACAO E EXPORTACAO LTDA EPP,0.0,0.000000,81,PERFURADORES ( PRF )
37524,38228,BALDE 8L PT FORTPLAST 1030.5,8751,FORT-PLAST INDUSTRIA E COMERCIO DE PLASTICOS E...,600.0,3.879582,159,BALDES PLASTICO ( BDE )
37525,38229,BALDE 10L PT FORTPLAST 1042.5,8751,FORT-PLAST INDUSTRIA E COMERCIO DE PLASTICOS E...,590.0,5.431415,159,BALDES PLASTICO ( BDE )


In [68]:
# Passo 1: Criar um dicionário {categoria: código_unico}
codigos_por_categoria = df_produto.groupby('Categoria')['Código da categoria'].first().to_dict()

# Passo 2: Substituir os códigos originais pelo código único
df_produto['Código da categoria'] = df_produto['Categoria'].map(codigos_por_categoria)

# Verificar resultados
print("Categorias únicas após padronização:", df_produto['Categoria'].nunique())
print("Códigos únicos após padronização:", df_produto['Código da categoria'].nunique())

Categorias únicas após padronização: 704
Códigos únicos após padronização: 704


In [69]:
df_produto.nunique()

Código Produto         36637
Produto                36637
Código da Marca          871
Marca                    871
Quantidade estoque      1406
Preço de custo         25550
Código da categoria      704
Categoria                704
dtype: int64

In [70]:
# Identificar códigos com multiplas categorias(Inconsistência)

inconsistencia_cat = df_produto.groupby('Código da categoria')['Categoria'].nunique()
cod_cat_problema = inconsistencia_cat[inconsistencia_cat > 1].index

print(f"\nCódigos com mais de um categoria associada: {len(cod_cat_problema)}")
print(df_produto[df_produto['Código da categoria'].isin(cod_cat_problema)].sort_values('Código da categoria'))


Códigos com mais de um categoria associada: 0
Empty DataFrame
Columns: [Código Produto, Produto, Código da Marca, Marca, Quantidade estoque, Preço de custo, Código da categoria, Categoria]
Index: []


In [71]:
# Caso 1: Códigos sem categoria associada
codigos_sem_categoria = df_produto[df_produto['Categoria'].isnull()]['Código da categoria'].nunique()
print(f"\nCódigos únicos sem categoria: {codigos_sem_categoria}")

# Caso 2: Categorias sem código associado
categorias_sem_codigo = df_produto[df_produto['Código da categoria'].isnull()]['Categoria'].nunique()
print(f"Categorias únicas sem código: {categorias_sem_codigo}")


Códigos únicos sem categoria: 0
Categorias únicas sem código: 0


In [72]:
df_produto

,Código Produto,Produto,Código da Marca,Marca,Quantidade estoque,Preço de custo,Código da categoria,Categoria
0,1,LAPIS Nº2 HB PT C/144 SERELEPE ECOLE,1,MARIA GENI DISTRIB. DE ART. DE PAP. LTDA,0.0,9.000000,708,LAPIS ( LAP )
1,3,CANETA HIDRO PONTA FINA PT CIS,1272,COMERCIO E IMPORTACAO SERTIC LTDA,0.0,1.480000,35011,CANETA HIDROGRAFICA (CNT)
2,4,BANDEIRINHAS JUNINAS MEGA,1,MARIA GENI DISTRIB. DE ART. DE PAP. LTDA,0.0,2.739700,359,BANDEIRAS ( BDJ )
3,5,FITA IMP EPSON FX890 MASTERPRINT,4281,GRUPO MASTER COMERCIO IMPORTACAO E EXPORTACAO ...,61.0,10.545270,3,FITAS PARA IMPRESSORAS ( FTA )
4,6,ACUCAR COMUM,1,MARIA GENI DISTRIB. DE ART. DE PAP. LTDA,0.0,2.500000,199,AÇÚCAR
...,...,...,...,...,...,...,...,...
37522,38226,PISTOLA P/APLICAR COLA QUENTE PQ FUTURO,8793,FUTURO BRASIL IMPORTACAO E EXPORTACAO LTDA EPP,0.0,0.000000,206,PISTOLA ( PIS )
37523,38227,PERFURADOR METAL 2F 20FL FUTURO,8793,FUTURO BRASIL IMPORTACAO E EXPORTACAO LTDA EPP,0.0,0.000000,81,PERFURADORES ( PRF )
37524,38228,BALDE 8L PT FORTPLAST 1030.5,8751,FORT-PLAST INDUSTRIA E COMERCIO DE PLASTICOS E...,600.0,3.879582,159,BALDES PLASTICO ( BDE )
37525,38229,BALDE 10L PT FORTPLAST 1042.5,8751,FORT-PLAST INDUSTRIA E COMERCIO DE PLASTICOS E...,590.0,5.431415,159,BALDES PLASTICO ( BDE )


In [73]:
estoq = df_produto.pop('Quantidade estoque')
df_produto['Quantidade estoque'] = estoq.astype(int)
preco = df_produto.pop('Preço de custo')
df_produto['Preço de custo'] = preco.round(2)


In [74]:
df_produto.describe()

,Código Produto,Código da Marca,Código da categoria,Quantidade estoque,Preço de custo
count,36637.000000,36637.000000,36637.000000,36637.000000,36637.000000
mean,18916.077272,4059.969457,2307.805825,94.076617,23.115083
std,10958.999948,20129.560751,8162.159959,1208.399580,436.649604
min,1.000000,1.000000,1.000000,-4.000000,0.000000
25%,9434.000000,995.000000,75.000000,0.000000,2.500000
50%,18821.000000,2712.000000,215.000000,0.000000,6.760000
75%,28281.000000,7157.000000,519.000000,11.000000,16.520000
max,38230.000000,901455.000000,35157.000000,166112.000000,79000.000000


In [75]:
df_produto.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36637 entries, 0 to 37526
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Código Produto       36637 non-null  int64  
 1   Produto              36637 non-null  object 
 2   Código da Marca      36637 non-null  int64  
 3   Marca                36637 non-null  object 
 4   Código da categoria  36637 non-null  int64  
 5   Categoria            36637 non-null  object 
 6   Quantidade estoque   36637 non-null  int64  
 7   Preço de custo       36637 non-null  float64
dtypes: float64(1), int64(4), object(3)
memory usage: 2.5+ MB


In [76]:
df_estoque.to_excel("bases/bases_limpas/base_estoque_limpo.xlsx", index=False)